In [23]:
import sys
import os
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from libs.signals import signals

btc_df = signals.technical_indicator_signal('btc')
btc_df.head()

,close,ewma_x,macd,bollinger,rsi,psar,vwap,signal
Date,,,,,,,,
2019-01-18 18:00:00,3683.0,-1,-1.0,-1,NaN,-1,1,-3.0
2019-01-19 18:00:00,3530.0,-1,-1.0,-1,1.0,-1,1,-2.0
2019-01-20 18:00:00,3533.4,-1,-1.0,-1,1.0,-1,0,-3.0
2019-01-21 18:00:00,3575.8,-1,-1.0,-1,1.0,-1,0,-3.0
2019-01-22 18:00:00,3556.1,-1,-1.0,-1,1.0,-1,1,-2.0


In [2]:
signals.technical_indicator_signal('eth').tail()

,close,ewma_x,macd,bollinger,rsi,psar,vwap,signal
Date,,,,,,,,
2021-01-02 18:00:00,979.86,1,1.0,-1,-1.0,1,0,1.0
2021-01-03 18:00:00,1042.52,1,1.0,-1,-1.0,1,0,1.0
2021-01-04 18:00:00,1103.41,1,1.0,-1,-1.0,1,0,1.0
2021-01-05 18:00:00,1211.85,1,1.0,-1,-1.0,1,0,1.0
2021-01-06 18:00:00,1246.93,1,1.0,-1,-1.0,1,0,1.0


In [27]:
from datetime import timedelta

In [32]:
# technical_indicator_signal(asset):

'''
Input: Asset symbol - 'btc', 'eth', 'ltc', 'dot', 'xmr', 'xdg', 'xlm', 'xrp', 'zec', 'nano', 'trx', 'bch', 'xtz', 'ada','oxt'
Returns:  Dataframe of technial indicator signals

NOTICE: When analyzing on the daily timeframe or greater, VWAP will not apply as it is ONLY an intraday indicator.
'''
asset = 'btc'

# create a dataframe to house the technical trading signals in
asset_df = kraken_data(asset)
technical_signals = pd.DataFrame({
    'close': asset_df.Close,
    'ewma_x': signals.ewma_crossover(asset_df).signal,
    'macd': signals.macd(asset_df).signal,
    'bollinger': signals.b_band(asset_df).signal,
    'rsi': signals.rsi(asset_df).signal,
    'psar': signals.psar(asset_df).signal,
    'vwap': signals.vwap(asset_df).signal 
})

# since VWAP won't work on daily time intervals and greater, we need to check the interval to see if we should include vwap as a column or not

daily_seconds = 86400
delta_seconds = timedelta.total_seconds(technical_signals.index[1] - technical_signals.index[0])
interval = delta_seconds / daily_seconds

if interval >= 1:
    # if the interval is greater than or equal to 1, then do not include VWAP
    technical_signals = technical_signals.drop(columns='vwap')

# sum the various technical signals together to return a trade 'grade' or signal
technical_signals['signal'] = technical_signals.drop(columns=['close']).sum(axis='columns')


technical_signals.tail()

,close,ewma_x,macd,bollinger,rsi,psar,signal
Date,,,,,,,
2021-01-02 18:00:00,33082.7,1,1.0,-1,-1.0,1,1.0
2021-01-03 18:00:00,32048.0,1,1.0,-1,-1.0,-1,-1.0
2021-01-04 18:00:00,34050.0,1,1.0,-1,-1.0,-1,-1.0
2021-01-05 18:00:00,36876.7,1,1.0,-1,-1.0,1,1.0
2021-01-06 18:00:00,39308.8,1,1.0,-1,-1.0,1,1.0
